In [ ]:
# Verbindung zum Google Drive aufbauen
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Auswahl der Tensorflow Version
%tensorflow_version 1.15.3
import os

# Installation von tf_slim
!pip install tf_slim

# Clone des models Repository
os.chdir('/content/drive/My Drive')
!git clone --quiet https://github.com/tensorflow/models.git

# Installation weiterer Pakete
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools

# Ausführung des Setup
os.chdir('/content/drive/My Drive/models/research')
!python setup.py build
!python setup.py install
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/object_detection

# Protoc Variable setzen
os.chdir('/content/drive/My Drive/models/research')
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/models/research/:/content/drive/My Drive/models/research/slim/'

# Test ob Installation erfolgreich
!python object_detection/builders/model_builder_test.py


In [ ]:
# Erstellung des quantized TFLite Model 
tflite_convert \
--graph_def_file="Pfad zu dem vorab erstellten Frozen-Graph" \ 
--output_file=tflite/detect_quant.tflite \ 
--output_format=TFLITE \
--input_shapes=1,640,640,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'  \
--inference_type=QUANTIZED_UINT8 \
--mean_values=128 \
--std_dev_values=127 \
--change_concat_input_ranges=false \
--allow_custom_ops